In [1]:
import pandas as pd
import numpy as np
import os
import pandas as pd
import numpy as np
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interactive, widgets, interact
import ipywidgets as widgets
#from pivottablejs import pivot_ui
import warnings
warnings.simplefilter('ignore')
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df_20210221 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/ranking_history/21-2.csv',header=6)
# Replace empty landing pages
df_20210221.replace(r'\s+',np.nan,regex=True).replace('',np.nan, inplace=True)
# Add date to columns, date should be changed manually
df_20210221['Datum'] = '2021-02-21'
# To avoid a warning
pd.options.mode.chained_assignment = None 
# Date to datetime64 Dtype
df_20210221['Datum'] =  pd.to_datetime(df_20210221['Datum'], format='%Y%m%d', infer_datetime_format=True)
# Rename columns
df_20210221.rename(columns={'tags':'Tags','*.expert.nl/*_20210221': 'Rankings_Expert', '*.expert.nl/*_20210221_landing': 'Expert_url','*.bcc.nl/*_20210221': 'Rankings_BCC', '*.bcc.nl/*_20210221_landing': 'BCC_url','*.coolblue.nl/*_20210221': 'Rankings_Coolblue', '*.coolblue.nl/*_20210221_landing': 'Coolblue_url','*.mediamarkt.nl/*_20210221': 'Rankings_Mediamarkt', '*.mediamarkt.nl/*_20210221_landing': 'Mediamarkt_url','*.bol.com/*_20210221': 'Rankings_Bol.com', '*.bol.com/*_20210221_landing': 'Bol.com_url','Search Volume': 'Search_Volume'}, inplace=True)
# Define which columns will be used
columns_to_use = df_20210221[['Datum','Keyword','Tags','Rankings_Expert','Expert_url','Rankings_BCC','BCC_url','Rankings_Coolblue','Coolblue_url','Rankings_Mediamarkt','Mediamarkt_url','Rankings_Bol.com','Bol.com_url','CPC','Search_Volume']]
columns_to_use.to_csv('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/use_cols/20210221.csv')

In [3]:
# Combine all files from directory
files = [file for file in os.listdir('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/use_cols/')]
workfile = pd.DataFrame()
for file in files:    
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/use_cols/'+ file)    
    workfile = pd.concat([workfile, df])
workfile.dropna(subset= ["Keyword"], inplace=True)
#workfile.drop(workfile.columns[1],axis=1,inplace=True)
workfile.replace('-',np.nan, inplace=True)
workfile['Rankings_Expert']=workfile['Rankings_Expert'].astype(float)
workfile['Rankings_BCC']=workfile['Rankings_BCC'].astype(float)
workfile['Rankings_Bol.com']=workfile['Rankings_Bol.com'].astype(float)
workfile['Rankings_Mediamarkt']=workfile['Rankings_Mediamarkt'].astype(float)
workfile['Rankings_Coolblue']=workfile['Rankings_Coolblue'].astype(float)
workfile.drop('Unnamed: 0', axis=1, inplace=True)
workfile.replace(r'\s+',np.nan,regex=True).replace('',np.nan, inplace=True)

In [4]:
df_vis_expert = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/visibility_competitors/vis_expert.xlsx')
df_vis_calc = workfile.merge(df_vis_expert,on='Rankings_Expert', how='outer')
df_vis_calc['Visibility_Expert'] = df_vis_calc.Visibility_Expert.astype(float)
df_vis_bcc = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/visibility_competitors/vis_bcc.xlsx')
df_vis_calc2 = df_vis_calc.merge(df_vis_bcc,on='Rankings_BCC', how='outer')
df_vis_calc2['Visibility_BCC'] = df_vis_calc2.Visibility_BCC.astype(float)
df_vis_coolblue = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/visibility_competitors/vis_coolblue.xlsx')
df_vis_calc3 = df_vis_calc2.merge(df_vis_coolblue,on='Rankings_Coolblue', how='outer')
df_vis_calc3['Visibility_Coolblue'] = df_vis_calc3.Visibility_Coolblue.astype(float)
df_vis_mediamarkt = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/visibility_competitors/vis_mediamarkt.xlsx')
df_vis_calc4 = df_vis_calc3.merge(df_vis_mediamarkt,on='Rankings_Mediamarkt', how='outer')
df_vis_calc4['Visibility_Mediamarkt'] = df_vis_calc4.Visibility_Mediamarkt.astype(float)
df_vis_bol = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/visibility_competitors/vis_bol.xlsx')
df_vis_calc5 = df_vis_calc4.merge(df_vis_bol,on='Rankings_Bol.com', how='outer')
df_vis_calc5['Visibility_Bol.com'] = df_vis_calc5['Visibility_Bol.com'].astype(float)

In [5]:
# Rename file & remove empty rows
df = df_vis_calc5.dropna(subset=['Keyword'])
# Optional: some sorting stuff
df.sort_values('Search_Volume', ascending=False, inplace=True)
# Optional: reset index
df.reset_index()
visibility = df[['Datum','Keyword','Tags','Visibility_Expert','Visibility_Coolblue','Visibility_Mediamarkt','Visibility_Bol.com','Visibility_BCC']]
visibility.head(3)
visibility.fillna(0, inplace=True)
visibility.to_csv('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/workfile/visibility.csv', index=False)
ranking = df[['Datum','Keyword','Search_Volume','Tags','Rankings_Expert','Rankings_Coolblue','Rankings_Mediamarkt','Rankings_Bol.com','Rankings_BCC']]
ranking.head(3)
ranking.fillna(0, inplace=True)
ranking.to_csv('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/workfile/ranking.csv', index=False)
vis = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/workfile/visibility.csv')
rnk = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/workfile/ranking.csv')

In [6]:
vis = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/workfile/visibility.csv')
rnk = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/workfile/ranking.csv')
vis.fillna(0, inplace=True)
# vis.drop('Unnamed: 0', axis=1, inplace=True)
vis.groupby(['Tags']).mean().reset_index()
vis['Datum'] =  pd.to_datetime(visibility['Datum'], format='%Y%m%d', infer_datetime_format=True)
pivot = vis.groupby(['Datum']).mean()
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SEO_reporting/workfile/visibility.csv')
dataset.fillna(0, inplace=True)
dataset['Datum'] =  pd.to_datetime(dataset['Datum'], format='%Y%m%d', infer_datetime_format=True)
rnk.fillna(0, inplace=True)
rnk['Datum'] =  pd.to_datetime(rnk['Datum'], format='%Y%m%d', infer_datetime_format=True)
rnk[rnk.eq(0)] = np.nan

In [7]:
tags = widgets.Dropdown(options=['All'] + list(dataset['Tags'].unique()),value='All',description='Category:',)
def plotit(tags):
    if 0 > 1:
        print('Nothing to show')  
    else: 
        dataset2 = dataset.copy()
        if tags != 'All':
            dataset2 = dataset2[dataset2.Tags == tags]
            #plt.style.use('Solarize_Light2')
            plt.show()
        
        if len(dataset2) > 0:
            dataset2.groupby(['Datum']).mean().plot(kind='line',figsize=(14,7))
            plt.xlabel('Period', fontsize=14)
            plt.ylabel('Visibility in Google Organic Search', fontsize=14)
            plt.title('Search Visibility in Google for {}'.format(tags), fontsize=20)
            #plt.style.use('Solarize_Light2')
            plt.show()
        else: 
            print('Nothing to show')
interactive(plotit, tags=tags)

interactive(children=(Dropdown(description='Category:', options=('All', 'nintendo', 'playstation', 'chromecast…